In [1]:
import numpy as np

In [2]:
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

In [3]:
def s1(n):
    A = (
        + np.diag(np.ones(n)*5, 0)
        + np.diag(np.ones(n-1)*-1, -1)
        + np.diag(np.ones(n-1)*-1, 1)
    )
    b = np.ones(n)*3
    b[[0, -1]] = 4
    return A, b
s1(5)

(array([[5.00, -1.00, 0.00, 0.00, 0.00],
        [-1.00, 5.00, -1.00, 0.00, 0.00],
        [0.00, -1.00, 5.00, -1.00, 0.00],
        [0.00, 0.00, -1.00, 5.00, -1.00],
        [0.00, 0.00, 0.00, -1.00, 5.00]]),
 array([4.00, 3.00, 3.00, 3.00, 4.00]))

In [4]:
def s2(n):
    A = (
        + np.diag(np.ones(n)*5, 0)
        + np.diag(np.ones(n-1)*-1, -1)
        + np.diag(np.ones(n-3)*-1, -3)
        + np.diag(np.ones(n-1)*-1, 1)
        + np.diag(np.ones(n-3)*-1, 3)
    )
    b = np.ones(n)
    b[[1, 2, -2, -3]] = 2
    b[[0, -1]] = 3
    return A, b
s2(7)

(array([[5.00, -1.00, 0.00, -1.00, 0.00, 0.00, 0.00],
        [-1.00, 5.00, -1.00, 0.00, -1.00, 0.00, 0.00],
        [0.00, -1.00, 5.00, -1.00, 0.00, -1.00, 0.00],
        [-1.00, 0.00, -1.00, 5.00, -1.00, 0.00, -1.00],
        [0.00, -1.00, 0.00, -1.00, 5.00, -1.00, 0.00],
        [0.00, 0.00, -1.00, 0.00, -1.00, 5.00, -1.00],
        [0.00, 0.00, 0.00, -1.00, 0.00, -1.00, 5.00]]),
 array([3.00, 2.00, 2.00, 1.00, 2.00, 2.00, 3.00]))

In [5]:
def s3(n):
    A = np.random.uniform(0, 1, size=(n, n))
    A += np.diag(np.ones(n))*(n+1)
    b = np.arange(1, n+1)
    return A, b
s3(10)

(array([[11.74, 0.37, 0.21, 0.50, 0.87, 0.16, 0.32, 0.33, 0.16, 0.40],
        [0.86, 11.65, 0.58, 0.29, 0.22, 0.24, 0.99, 0.13, 0.70, 0.07],
        [0.69, 0.77, 11.92, 0.52, 0.53, 0.18, 0.61, 0.53, 0.20, 0.07],
        [0.73, 0.17, 0.71, 11.81, 0.08, 0.43, 0.57, 0.57, 0.10, 0.34],
        [0.05, 0.33, 0.14, 0.04, 11.54, 0.86, 0.28, 0.59, 0.93, 0.60],
        [0.15, 0.77, 0.70, 0.29, 0.38, 11.44, 0.57, 0.84, 0.63, 0.17],
        [0.04, 0.18, 0.11, 0.38, 0.65, 0.27, 11.70, 0.52, 0.05, 0.77],
        [0.87, 0.65, 0.69, 0.92, 0.12, 0.99, 0.81, 11.83, 0.26, 0.06],
        [0.67, 0.36, 0.82, 0.61, 0.58, 0.15, 0.86, 0.14, 11.35, 0.82],
        [0.50, 0.98, 0.97, 0.63, 0.44, 0.83, 0.05, 0.52, 0.64, 11.75]]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]))

# Metoda Jacobi

In [6]:
def jacobi(A, b, iter=100, thresh=10**-10):
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    n = A.shape[0]
    
    d = np.diag(A)
    coef = np.concatenate([-b.reshape(-1, 1), A - np.diag(d)], 1) / -d.reshape(-1, 1)

    converged = False
    x = np.zeros(n)
    for i in range(iter):
        xp = x
        xe = np.concatenate([np.ones(1), xp])
        x = np.sum(coef * xe, 1)

        err = np.abs(np.mean(xp - x))
        if err < thresh:
            print(f"a convers in {i} iteratii")
            converged = True
            break
    if not converged:
        print("nu a convers")
    return x


n = 10

print("s1")
A, b = s1(n)
print(jacobi(A, b))
print(np.linalg.solve(A, b))

print("s2")
A, b = s2(n)
print(jacobi(A, b))
print(np.linalg.solve(A, b))

s1
a convers in 24 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
s2
a convers in 56 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]


# Metoda Gauss-Seidel

In [7]:
def gauss_seidel(A, b, iter=100, thresh=10**-10):
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    n = A.shape[0]
    
    d = np.diag(A)
    coef = np.concatenate([-b.reshape(-1, 1), A - np.diag(d)], 1) / -d.reshape(-1, 1)

    converged = False
    x = np.zeros(n)
    for i in range(iter):
        xp = x
        xe = np.concatenate([np.ones(1), xp])
        for j in range(n):
            xe[j+1] = np.sum(coef[j] * xe)
        x = xe[1:]

        err = np.abs(np.mean(xp - x))
        if err < thresh:
            print(f"a convers in {i} iteratii")
            converged = True
            break
    if not converged:
        print("nu a convers")
    return x


n = 10

print("s1")
A, b = s1(n)
print(gauss_seidel(A, b))
print(np.linalg.solve(A, b))

print("s2")
A, b = s2(n)
print(gauss_seidel(A, b))
print(np.linalg.solve(A, b))

s1
a convers in 15 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
s2
a convers in 31 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]


# Metoda SOR

In [8]:
def get_omega(A):
    D = np.diag(np.diag(A))
    L = np.tril(A) - D
    U = np.triu(A) - D
    
    T = np.matmul(np.linalg.inv(D), (L+U))
    spectral = np.max(np.abs(np.linalg.eigvals(T)))
    assert spectral < 1
    omega = 2 / (1 + np.sqrt(1 - spectral**2))
    return omega

In [9]:
def sor(A, b, iter=100, thresh=10**-10, omega=0.7):
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    n = A.shape[0]

    x = np.zeros(n)
    converged = False
    for i in range(iter):
        xp = x.copy()
        for j in range(n):
            x[j] = (
                + (1 - omega)*xp[j]
                + (omega / A[j, j]) * (
                    b[j] - np.sum(A[j, :j]*x[:j]) - np.sum(A[j, j+1:]*xp[j+1:])
                )
            )

        err = np.abs(np.mean(xp - x))
        if err < thresh:
            print(f"a convers in {i} iteratii")
            converged = True
            break
    if not converged:
        print("nu a convers")
    return x


n = 10

print("s1")
A, b = s1(n)
print(sor(A, b, omega=get_omega(A)))
print(np.linalg.solve(A, b))

print("s2")
A, b = s2(n)
print(sor(A, b, omega=get_omega(A)))
print(np.linalg.solve(A, b))

s1
0.3837971894457989
a convers in 11 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
s2
0.676138620597146
a convers in 19 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]


In [10]:
A, b = s3(10)

print(np.linalg.solve(A, b))
print(jacobi(A, b))
print(gauss_seidel(A, b))
print(sor(A, b, omega=get_omega(A)))

[-0.05 -0.01 0.15 0.22 0.31 0.35 0.43 0.62 0.66 0.76]
a convers in 24 iteratii
[-0.05 -0.01 0.15 0.22 0.31 0.35 0.43 0.62 0.66 0.76]
a convers in 10 iteratii
[-0.05 -0.01 0.15 0.22 0.31 0.35 0.43 0.62 0.66 0.76]
0.387905297606404
a convers in 10 iteratii
[-0.05 -0.01 0.15 0.22 0.31 0.35 0.43 0.62 0.66 0.76]
